# Imports 

In [1]:
import numpy as np, pandas as pd 
pd.set_option('display.max_columns', None)
import sqlite3, sqlalchemy, sys, os  

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('functions.py'), '..', '..', 'PY')))
import functions  

seasons = functions.generate_seasons(1973, 2024)
# seasons = functions.generate_seasons(2007, 2024)

# Convert CSVs to Dataframes 

In [2]:
engine = sqlalchemy.create_engine("sqlite:///../../DB/ballbase.db") 
def clean_player(name): 
    i = name.find('*')   
    name = name[:i] if i > -1 else name 
    return name 
for season in seasons:
    dir = f"../../static/CSV/{season}/" 

    names = ['pergame_'+season, 'advanced_'+season, 'per100_'+season, 'playbyplay_'+season, 'totals_'+season]  
    dfs = {} 

    invalid = ['-9999', 'Unnamed', 'Rk', 'Player-additional']

    for name in names:
        dfs[name] = pd.read_csv(dir+name+'.csv') 
        for inv in invalid:
            for col in dfs[name].columns:
                if inv in col or col in invalid:

                    dfs[name] = dfs[name].drop([col], axis=1)
                    
    dfs['totals_'+season] = dfs['totals_'+season].loc[:,['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'MP', 'FG', 'FGA', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB']] 
    dfs['totals_'+season].columns = 'T' + dfs['totals_'+season].columns 
    dfs['totals_'+season]['Player'] =  dfs['pergame_'+season]['Player']

    def combine_dfs(dfs, names):
        curr = None
        for name in names:
            if curr is None: 
                curr = dfs[name]
            else:
                diff = dfs[name].columns.difference(curr.columns).tolist()
                diff.append('Player') 
                curr = pd.merge(curr, dfs[name].loc[:,diff], on = 'Player') 
        return curr     

    master = combine_dfs(dfs, names) 
    master['Player'] = master['Player'].apply(lambda x: clean_player(x)) 

    for name in names: 
        name_temp = name 
        if name == 'pergame_'+season: 
            name_temp = 'averages_'+season 
        dfs[name]['Player'] = dfs[name]['Player'].apply(lambda x: clean_player(x))
        dfs[name].to_sql(name_temp, con = engine, if_exists = 'replace', index = False)  
        
    master.to_sql('master_'+season, con=engine, if_exists='replace', index=False) 
     

FileNotFoundError: [Errno 2] No such file or directory: '../../static/CSV/1973_74/pergame_1973_74.csv'

# Output Tables (Multiple Seasons) 

In [8]:
engine = sqlalchemy.create_engine("sqlite:///../../DB/ballbase.db") 
for season in seasons:
    
    master = pd.read_sql("master_"+season, con=engine).fillna(0).drop("Tm", axis=1)
    master.columns 
    averages = master[['Player', 'Pos', 'Age', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'MP', 'FGA', 'FG%', '3PA', '3P%', 'FTA', 'FT%', 'PF']] 
    totals = master.loc[:, ['Player', 'TPTS', 'TSTL',
        'TTOV', 'TTRB', 'T2P', 'T2PA', 'T3PA', 'TAST', 'TBLK', 'TDRB',
        'TFG', 'TFGA', 'TFT', 'TFTA', 'TMP', 'TORB', 'TPF']]
    advanced = master.loc[:, ['Player', 'DRtg', 'ORtg',
        '3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
        'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP',
        'WS', 'WS/48']] 


    contents = [averages, totals, advanced] 
    # Create HTML table version 
    for i in range(len(contents)):
        contents[i] = contents[i].to_html(table_id="table", classes = "table table-hover table-stripped table-bordered table-striped", border=1, index=False)
        contents[i] = contents[i].replace('<table border="1" class="dataframe table table-hover table-stripped table-bordered table-striped" id="table">', '<table border="1" class="dataframe table table-hover table-stripped table-bordered" style="width:100%" id="table">')
    # Create page content 
    for i in range(len(contents)):
        contents[i] = f"""
{{%extends 'TableBase.html'%}}
{{% block table %}} 

{contents[i]}
{{% endblock %}}
"""
    def write_to_file(path, content): 
        file = open(path, "w", encoding="utf-8") 
        file.write(content) 
        file.close 


    file_names=[f"Averages_{season}.html", f"Totals_{season}.html", f"Advanced_{season}.html"] 

    for i in range(len(contents)): 
        path = f"../../templates/Tables/{file_names[i]}" 
        write_to_file(path, contents[i]) 

# Output Season Buttons 

In [5]:
season_labs = functions.generate_seasons(1973, 2024)  
temp = []

season_button_content = ""
for season in season_labs: 
    ssn_str = season.replace('_', '-') 
    season_button_content += f'<button onclick="seasonSelect(\'{season}\')" class="dropdown-button"> {ssn_str} </button>' + '\n'  
    temp.append(f'<button onclick="seasonSelect(\'{season}\')" class="dropdown-button"> {ssn_str} </button>') 
for i in range(len(temp)-1, -1, -1):
    print(temp[i]) 
# file_b = open("../../templates/Other/Season-buttons.html", "w", encoding="utf-8") 
# file_b.write(season_button_content) 
# file_b.close()  
# print(season_button_content)

<button onclick="seasonSelect('2023_24')" class="dropdown-button"> 2023-24 </button>
<button onclick="seasonSelect('2022_23')" class="dropdown-button"> 2022-23 </button>
<button onclick="seasonSelect('2021_22')" class="dropdown-button"> 2021-22 </button>
<button onclick="seasonSelect('2020_21')" class="dropdown-button"> 2020-21 </button>
<button onclick="seasonSelect('2019_20')" class="dropdown-button"> 2019-20 </button>
<button onclick="seasonSelect('2018_19')" class="dropdown-button"> 2018-19 </button>
<button onclick="seasonSelect('2017_18')" class="dropdown-button"> 2017-18 </button>
<button onclick="seasonSelect('2016_17')" class="dropdown-button"> 2016-17 </button>
<button onclick="seasonSelect('2015_16')" class="dropdown-button"> 2015-16 </button>
<button onclick="seasonSelect('2014_15')" class="dropdown-button"> 2014-15 </button>
<button onclick="seasonSelect('2013_14')" class="dropdown-button"> 2013-14 </button>
<button onclick="seasonSelect('2012_13')" class="dropdown-button"